In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import datetime

def load():
    # data_dir = "/home/asia/Documents/projects/PyCharm/IUM/data/data_v3/"
    data_dir = "/content/drive/MyDrive/Data/IUM/V3/"
    data = {}
    data["deliveries"] = pd.read_json(data_dir + "deliveries.jsonl", lines = True)
    data["sessions"] = pd.read_json(data_dir + "sessions.jsonl", lines = True)
    data["products"] = pd.read_json(data_dir + "products.jsonl", lines = True)
    data["users"] = pd.read_json(data_dir + "users.jsonl", lines = True)
    return data

def preprocess_time(deliveries: pd.DataFrame):
    deliveries["purchase_timestamp"] = pd.to_datetime ( deliveries["purchase_timestamp"], format='%Y-%m-%dT%H:%M', errors="coerce")
    deliveries['delivery_timestamp'] = pd.to_datetime ( deliveries["delivery_timestamp"], format='%Y-%m-%dT%H:%M:%S', errors="coerce")
    return deliveries


def merge(data: {}) -> pd.DataFrame:
    s = data["sessions"]
    d = data["deliveries"]
    p = data["products"]
    u = data["users"]
    purchases = s[s["purchase_id"].notnull()].copy()
    merged = pd.merge(purchases, d, on='purchase_id', how = "outer")
    merged2 = pd.merge(merged, u, on="user_id", how = "outer")
    merged_full = pd.merge(merged2, p, on="product_id", how = "left")
    return merged_full

def select_columns (merged):
    to_select = ["product_id", "offered_discount", "purchase_timestamp", "delivery_timestamp", "delivery_company", "city", "street", "product_name", "category_path", "price"]
    return merged[to_select]

In [58]:
# load data, make sure timestamps are of datetime type and merge data
data = load()
data["deliveries"] = preprocess_time(data["deliveries"])
merged = merge(data)
merged.head()

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company,name,city,street,product_name,category_path,price
0,100002.0,2021-04-01 15:52:47,102,1234.0,BUY_PRODUCT,0.0,20001.0,2021-04-01 15:52:47,2021-04-02 14:06:34.432964,620.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Sony DVP-SR760,Sprzęt RTV;Video;Odtwarzacze DVD,193.00
1,100003.0,2021-04-01 02:26:23,102,1289.0,BUY_PRODUCT,15.0,20002.0,2021-04-01 02:26:23,2021-04-02 10:45:27.707128,360.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Philips SDV5225,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,129.00
2,100004.0,2021-01-03 05:11:17,102,1009.0,BUY_PRODUCT,0.0,20003.0,2021-01-03 05:11:17,2021-01-04 15:43:07.635371,620.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00
3,100005.0,2021-02-17 12:59:54,102,1001.0,BUY_PRODUCT,5.0,20004.0,2021-02-17 12:59:54,2021-02-18 11:52:42.608746,620.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97
4,100006.0,2021-02-13 09:17:52,102,1057.0,BUY_PRODUCT,5.0,20005.0,2021-02-13 09:17:52,2021-02-15 08:59:28.973167,620.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Doom 3 (PC),Gry i konsole;Gry komputerowe,19.99


In [59]:
# drop that one row with user who didn't make a purchase
merged = merged.dropna()
merged.shape

(7287, 16)

In [60]:
def transform_columns(merged_df):
  merged_df["time_difference"] = merged_df["delivery_timestamp"] - merged_df["purchase_timestamp"]
  merged_df["time[min]"] = merged_df["time_difference"].apply(lambda x: int(round(x.total_seconds()/60)))
  merged_df = merged_df.drop(labels=["time_difference", "delivery_timestamp"], axis=1)

  merged_df["street_name"] = merged_df["street"].apply(lambda x: x.split(" ")[1])
  merged_df["street_name*num"] = merged_df["street"].apply(lambda x: x.split(" ")[1] + " " + x.split(" ")[2])
  merged_df = add_city_street(merged_df)
  merged_df = merged_df.drop(labels=["street"], axis=1)

  merged_df["purchase_day_of_week"] = merged_df["purchase_timestamp"].apply(lambda x: x.dayofweek)
  merged_df["purchase_day"] = merged_df["purchase_timestamp"].apply(lambda x: x.replace(second=0, minute=0, hour=0))
  merged_df["purchase_minute"] = merged_df["purchase_timestamp"].apply(lambda x: x.replace(second=0, month=1, day=1))
  merged_df = merged_df.drop(labels=["purchase_timestamp"], axis=1)
  return merged_df

def add_city_street(merged_df):
  cities = merged_df["city"].to_list()
  streets = merged_df["street_name"].to_list()

  city_street = [cities[i] + "/" + streets[i] for i in range(len(cities))]

  merged_df["city*stret_name"] = city_street
  return merged_df

merged = select_columns(merged)
merged = transform_columns(merged)


In [61]:
merged.head()

,product_id,offered_discount,delivery_company,city,product_name,category_path,price,time[min],street_name,street_name*num,city*stret_name,purchase_day_of_week,purchase_day,purchase_minute
0,1234.0,0.0,620.0,Police,Sony DVP-SR760,Sprzęt RTV;Video;Odtwarzacze DVD,193.00,1334,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,3,2021-04-01,2021-01-01 15:52:00
1,1289.0,15.0,360.0,Police,Philips SDV5225,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,129.00,1939,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,3,2021-04-01,2021-01-01 02:26:00
2,1009.0,0.0,620.0,Police,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,2072,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,6,2021-01-03,2021-01-01 05:11:00
3,1001.0,5.0,620.0,Police,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97,1373,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,2,2021-02-17,2021-01-01 12:59:00
4,1057.0,5.0,620.0,Police,Doom 3 (PC),Gry i konsole;Gry komputerowe,19.99,2862,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,5,2021-02-13,2021-01-01 09:17:00
